In [1]:
#This is for top transaction state wise data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/top/transaction/country/india/state"
Top_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'Entity_name':[], 'transaction_count':[], 'transaction_amount':[]}

for i in Top_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['districts']:
              Name=z['entityName']
              count=z['metric']['count']
              amount=z['metric']['amount']
              clm['Entity_name'].append(Name)
              clm['transaction_count'].append(count)
              clm['transaction_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Top_Trans_state=pd.DataFrame(clm)
print(Top_Trans_state)

                          State  Year  Quater               Entity_name  \
0     andaman-&-nicobar-islands  2018       1             south andaman   
1     andaman-&-nicobar-islands  2018       1                  nicobars   
2     andaman-&-nicobar-islands  2018       1  north and middle andaman   
3     andaman-&-nicobar-islands  2018       2             south andaman   
4     andaman-&-nicobar-islands  2018       2                  nicobars   
...                         ...   ...     ...                       ...   
8291                west-bengal  2024       4                    howrah   
8292                west-bengal  2024       4                     nadia   
8293                west-bengal  2024       4                     malda   
8294                west-bengal  2024       4                   hooghly   
8295                west-bengal  2024       4         paschim medinipur   

      transaction_count  transaction_amount  
0                  5688        1.256025e+07  
1      

In [2]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Top_Trans_state['State'] = Top_Trans_state['State'].str.strip().str.lower()
Top_Trans_state['State'] = Top_Trans_state['State'].replace(state_name_map)

In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Top_Transaction_State(
        State varchar(100),
        Year int,
        Quater int,
        Entity_name varchar(100),
        Transaction_count float,
        transaction_amount float)
        """
cursor.execute(query1)

In [4]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Top_Trans_state.index:
    row = Top_Trans_state.loc[index]
    row = row.values
    row = row[0],float(row[1]),int(row[2]),row[3],int(row[4]),float(row[5])
    data.append(row)
query2 = "insert into Top_Transaction_State values (%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [3]:
#doing data cleaning for Top_Trans_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Top_Trans_state.isna().sum()
Top_Trans_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['Entity_name']
dummy_col = ['Year','State','Quater']
encoder = LabelEncoder()
Top_Trans_state['encoded_entity_name'] = encoder.fit_transform(Top_Trans_state['Entity_name'])
Top_Trans_state = pd.get_dummies(Top_Trans_state,columns=dummy_col,dtype=int)
#third removing outliners
Top_Trans_state['transaction_count_std']=(Top_Trans_state['transaction_count']-Top_Trans_state['transaction_count'].mean())/Top_Trans_state['transaction_count'].std()
Top_Trans_state['transaction_amount_std']=(Top_Trans_state['transaction_amount']-Top_Trans_state['transaction_amount'].mean())/Top_Trans_state['transaction_amount'].std()
condition1 = Top_Trans_state['transaction_count_std']<=3
condition2 = Top_Trans_state['transaction_count_std']>=-3
condition3 = Top_Trans_state['transaction_amount_std']<=3
condition4 = Top_Trans_state['transaction_amount_std']>=-3
Top_Trans_state = Top_Trans_state[(condition1) & (condition2) & (condition3) & (condition4)]
print(Top_Trans_state)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8296 entries, 0 to 8295
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               8296 non-null   object 
 1   Year                8296 non-null   object 
 2   Quater              8296 non-null   int64  
 3   Entity_name         8296 non-null   object 
 4   transaction_count   8296 non-null   int64  
 5   transaction_amount  8296 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 389.0+ KB
                   Entity_name  transaction_count  transaction_amount  \
0                south andaman               5688        1.256025e+07   
1                     nicobars                528        1.139849e+06   
2     north and middle andaman                442        9.316631e+05   
3                south andaman               9395        2.394824e+07   
4                     nicobars               1120        3.072437e+06   


In [12]:
#plotting aggregated_transaction_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Top_Trans_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='transaction_count',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [4]:
#This is for top transaction data for 2018
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/top/transaction/country/india/2018"
Top_trans_2018_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'Quater':[],'Place_name':[], 'transaction_count':[], 'transaction_amount':[]}

for i in Top_trans_2018_list:
    p_i=path+"/"+i
    Data=open(p_i,'r')
    D=json.load(Data)
    for z in D['data']['districts']:
        Name=z['entityName']
        count=z['metric']['count']
        amount=z['metric']['amount']
        clm['Place_name'].append(Name)
        clm['transaction_count'].append(count)
        clm['transaction_amount'].append(amount)
        clm['Quater'].append(int(i.strip('.json')))
#Succesfully created a dataframe
Top_Trans_state=pd.DataFrame(clm)
print(Top_Trans_state)

    Quater                  Place_name  transaction_count  transaction_amount
0        1             bengaluru urban            8306513        1.416275e+10
1        1                        pune            3842929        5.378982e+09
2        1               central delhi            2997714        4.312145e+09
3        1                      mumbai            2671029        3.923117e+09
4        1                     chennai            2656230        4.281138e+09
5        1                   hyderabad            2545507        4.287224e+09
6        1                      jaipur            2417174        2.953087e+09
7        1                     birbhum            2125816        2.520486e+09
8        1                     kolkata            2043129        2.753348e+09
9        1                      bhopal            1704495        1.905474e+09
10       2             bengaluru urban           11916035        2.382473e+10
11       2                        pune            5162876       